In [4]:
import numpy as np


def target_mean_v1(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    for i in range(data.shape[0]):
        groupby_result = data[data.index != i].groupby([x_name], as_index=False).agg(
            ['mean', 'count'])
        result[i] = groupby_result.loc[
            groupby_result.index == data.loc[i, x_name], (y_name, 'mean')]
    return result


In [6]:
import pandas as pd
y = np.random.randint(2, size=(5000, 1))
x = np.random.randint(10, size=(5000, 1))
data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])
result_1 = target_mean_v1(data, 'y', 'x')
%timeit target_mean_v1(data, 'y', 'x')

1 loop, best of 3: 24.6 s per loop


In [7]:
from collections import defaultdict

def target_mean_v2(data, y_name, x_name):
    count = data.shape[0]
    result = np.zeros(count)
    sum_dict = defaultdict(int)
    count_dict = defaultdict(int)
    for i in range(count):
        sum_dict[data.loc[i, x_name]] += data.loc[i, y_name]
        count_dict[data.loc[i, x_name]] += 1

    for i in range(count):
        result[i] = (sum_dict[data.loc[i, x_name]] - data.loc[i, y_name]) / (
                count_dict[data.loc[i, x_name]] - 1)
    return result

In [9]:
result_2 = target_mean_v2(data, 'y', 'x')

diff = np.linalg.norm(result_1 - result_2)
print(diff)

%timeit target_mean_v2(data, 'y', 'x')

0.0
1 loop, best of 3: 229 ms per loop


In [10]:
def target_mean_v3(y_values, x_values):
    sum_dict = defaultdict(int)
    count_dict = defaultdict(int)

    for x_v, y_v in zip(x_values, y_values):
        sum_dict[x_v] += y_v
        count_dict[x_v] += 1

    result = [(sum_dict[x_v] - y_v)/(count_dict[x_v]-1) for x_v, y_v in zip(x_values, y_values)]
    return result

In [11]:
result_3 = target_mean_v3(data['y'].values, data['x'].values)

diff = np.linalg.norm(result_1 - result_3)
print(diff)

%timeit target_mean_v3(data['y'].values, data['x'].values)

0.0
100 loops, best of 3: 6.68 ms per loop
